In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [2]:
glass = pd.read_csv('glass.csv')
glass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
RI      214 non-null float64
Na      214 non-null float64
Mg      214 non-null float64
Al      214 non-null float64
Si      214 non-null float64
K       214 non-null float64
Ca      214 non-null float64
Ba      214 non-null float64
Fe      214 non-null float64
Type    214 non-null int64
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


In [3]:
# Dividir o dados para treinamento e teste
train, test = train_test_split(glass, test_size=.4, random_state=123)

print('train {0}'.format(train.shape))
print('test {0}'.format(test.shape))

X_train = train.drop(['Type'], axis=1)
y_train = train['Type']

X_test = test.drop(['Type'], axis=1)
y_test = test['Type']

print('X_train {0}'.format(X_train.shape))
print('y_train {0}'.format(y_train.shape))
print('X_test {0}'.format(X_test.shape))
print('y_test {0}'.format(y_test.shape))

train (128, 10)
test (86, 10)
X_train (128, 9)
y_train (128,)
X_test (86, 9)
y_test (86,)


In [4]:
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.71      0.74      0.72        23
           2       0.67      0.90      0.76        29
           3       0.00      0.00      0.00         6
           5       0.75      0.38      0.50         8
           6       1.00      0.50      0.67         4
           7       0.81      0.81      0.81        16

   micro avg       0.71      0.71      0.71        86
   macro avg       0.66      0.55      0.58        86
weighted avg       0.68      0.71      0.68        86



In [5]:
# K-fold cross validation
# k=10 sobre o set de treinamento, bagging
X = train.drop(['Type'], axis=1)
y = train.Type
scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')
print('accuracy:', scores)
print('avg accuracy:', scores.mean())

/home/jecs89/anaconda3/envs/tutorialConda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


accuracy: [0.75       0.73333333 0.73333333 0.78571429 0.71428571 0.5
 0.75       0.7        0.6        0.6       ]
avg accuracy: 0.6866666666666666


In [6]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# avaliando o algoritmo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.67      0.70      0.68        23
           2       0.68      0.72      0.70        29
           3       0.40      0.33      0.36         6
           5       0.83      0.62      0.71         8
           6       0.60      0.75      0.67         4
           7       0.93      0.88      0.90        16

   micro avg       0.71      0.71      0.71        86
   macro avg       0.69      0.67      0.67        86
weighted avg       0.71      0.71      0.71        86



In [7]:
# K-fold cross validation
# k=10 sobre o set de treinamento, clf
X = train.drop(['Type'], axis=1)
y = train.Type
scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print('accuracy:', scores)
print('avg accuracy:', scores.mean())

accuracy: [0.75       0.46666667 0.8        0.64285714 0.42857143 0.33333333
 0.66666667 0.7        0.6        0.5       ]
avg accuracy: 0.5888095238095238


/home/jecs89/anaconda3/envs/tutorialConda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
